In [1]:
import spacy
import json
import random
from spacy.util import minibatch, compounding
from spacy.training import Example
import pandas as pd

nlp = spacy.load('en_core_web_lg')
# from spacy.lang.en import English
# from spacy.pipeline import EntityRuler

In [ ]:
def load_data(file):
    data = pd.read_csv(file, encoding='latin1', index_col = False)
    return(data)

def save_data(file,data):
    with open(file, "w",encoding="utf-8") as f:
        json.dump(data, f, indent = 4)

def get_position(txt, str):
    start = txt.find(str)
    end = txt.find(str)+len(str)
    return start,end

data = load_data("tagged data.csv")

import numpy as np
data = data.replace(np.nan, '', regex=True)
data.head()



In [9]:
txt = "the Building is only 8 Years old and They are reusing the Bricks old."
print(txt)
get_position(txt,'8 Years old')

TRAIN_DATA = []
data1 = data


for i in range(len(data1)):
    entities = []
    text = data1['NOTES'][i].strip()
    if data1['entity_age'][i]!= '':
        start_char, end_char = get_position(text.lower(),data1['entity_age'][i].lower())
        if start_char >= 0: 
            entities = [(start_char, end_char, data1['category_age'][i])]
    if data1['entity_object'][i] != '':
        start_char1, end_char1 = get_position(text.lower(),data1['entity_object'][i].lower())
        if start_char1 >= 0:
            entities = entities + [(start_char1, end_char1, data1['category_object'][i])]
    if len(entities) > 0 :
        results = [text,{"entities": entities}]
    TRAIN_DATA = TRAIN_DATA + [results]
    

the Building is only 8 Years old and They are reusing the Bricks old.


In [10]:
# TRAIN_DATA = []
# with open("hpchapter1.txt","r",encoding="utf-8") as f:
#     text = f.read()
#     chapters = text.split("CHAPTER")[1:]
#     for chapter in chapters:
#         chapter_num, chapter_title = chapter.split("\n\n")[0:2]
#         segments = chapter.split("\n\n")[2:]
#         results = list()
#         for segment in segments:
#             segment = segment.replace("\n"," ").strip()
#             result = test_model(nlp,segment)
#             if result != []:
#                 results = results + [result]
#         TRAIN_DATA = TRAIN_DATA + results
print(len(TRAIN_DATA))


129


In [11]:
TRAIN_DATA[0]

['the Building is only 8 Years old and They are reusing the Bricks .',
 {'entities': [(21, 32, 'age'), (4, 12, 'building')]}]

In [6]:
save_data("train_data.json",TRAIN_DATA)
for a , b in TRAIN_DATA:
    print(b)

TRAIN_DATA[4]

{'entities': [(21, 32, 'age'), (4, 12, 'building')]}
{'entities': [(42, 45, 'old')]}
{'entities': [(36, 39, 'new'), (40, 44, 'building')]}
{'entities': [(60, 63, 'old'), (40, 44, 'building')]}
{'entities': [(2085, 2088, 'old'), (15, 19, 'roof')]}
{'entities': [(9, 21, 'age')]}
{'entities': [(16, 23, 'new'), (5, 9, 'building')]}
{'entities': [(46, 58, 'age'), (28, 36, 'building')]}
{'entities': [(14, 17, 'old'), (0, 8, 'building')]}
{'entities': [(378, 395, 'age'), (68, 73, 'equipment')]}
{'entities': [(266, 269, 'old'), (270, 278, 'building')]}
{'entities': [(266, 269, 'old'), (283, 287, 'plumbing')]}
{'entities': [(11, 14, 'old'), (15, 28, 'building')]}
{'entities': [(202, 213, 'age'), (53, 58, 'equipment')]}
{'entities': [(287, 300, 'age'), (264, 272, 'building')]}
{'entities': [(841, 854, 'age'), (818, 826, 'building')]}
{'entities': [(287, 300, 'age'), (264, 272, 'building')]}
{'entities': [(-1, 3, 'age'), (18, 23, 'building')]}
{'entities': [(14, 17, 'old'), (18, 23, 'building')]}

['the referenced Roof Leaks were the Result of one or more of the Following:-improper Repair Methods and materials;-deteriorated Condition of the Roof covering as a Result of normal Wear and Weathering;-workmanship Deficiencies associated with the Installation of the Roof Assembly; and - improper Maintenance of the Roof assembly.*the referenced stained and/or deteriorated Portions of Ceiling finishes within the Dining Room, Kitchen, Guest Bathroom and southeast Bedroom were not the Result of Storm - created Openings in the Roof Assembly attributable to Forces associated with severe Weather Events ( e.g. Wind damage).*based on the observed Damage, the Roof Leaks above Dining Room, Kitchen, Guest Bathroom and southeast Bedroom were long - Term, repetitive and ongoing over a Period of 1 Year or more prior to the reported Date of loss.*the preceding Opinions were based on the Following:-ghd?s Observations; including but not limited to the Following*the Age and Condition of the Roof Coverin

In [52]:
### create a function that will be used to train the model 
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank("en")
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner", last=True)
    
    for a , annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(other_pipes):
        optimizer = nlp.begin_training()
        
        for itn in range(iterations):
            print("starting iteration: " + str(itn))
            
            for itn in range(iterations):
                random.shuffle(TRAIN_DATA)
                losses = {}
                
                for batch in minibatch(TRAIN_DATA, size=8):
                    for text, annotations in batch:
                        doc = nlp.make_doc(text)
                        example = Example.from_dict(doc, annotations)
                        nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
                print(losses)
    return(nlp)

In [53]:
nlp = train_spacy(TRAIN_DATA, 50)

starting iteration: 0


/home/aditya/anaconda3/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "10022019160958 - Andrew Mrs Robinson came by the L..." with entities "[(450, 461, 'age'), (424, 441, 'equipment')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/home/aditya/anaconda3/lib/python3.6/site-packages/spacy/training/iob_utils.py:142: UserWarning: [W030] Some entities could not be aligned in the text "Insured had a Water Leak in the Basement of the in..." with entities "[(73, 77, 'new'), (77, 88, 'equipment')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
/home/aditya/anacond

{'ner': 968.9786353457198}
{'ner': 370.8292291259884}
{'ner': 337.19992058293485}
{'ner': 280.13045840570516}
{'ner': 305.66127186155}
{'ner': 313.5977996913116}
{'ner': 238.89524131350683}
{'ner': 227.60251107200776}
{'ner': 212.83772171041593}
{'ner': 185.28964041750936}
{'ner': 196.13715294099276}
{'ner': 155.2757759232402}
{'ner': 187.18618570763948}
{'ner': 180.1733643396007}
{'ner': 181.3009913880967}
{'ner': 157.3978552214491}
{'ner': 142.85882360416173}
{'ner': 146.37245480064814}
{'ner': 149.8418516556912}
{'ner': 131.87174024743993}
{'ner': 135.19770770039338}
{'ner': 144.4834116441908}
{'ner': 104.45527544368284}
{'ner': 123.74440581651939}
{'ner': 103.61858065055308}
{'ner': 85.50007351666791}
{'ner': 102.22053116157744}
{'ner': 95.8260392756772}
{'ner': 98.12627260870293}
{'ner': 101.32136627016928}
{'ner': 92.58247893579374}
{'ner': 71.18972932909472}
{'ner': 92.38997192835181}
{'ner': 96.99758432806436}
{'ner': 76.11178801726587}
{'ner': 86.32062632033183}
{'ner': 72.150

{'ner': 37.223608979336184}
{'ner': 51.909077522829776}
starting iteration: 6
{'ner': 35.627100501423186}
{'ner': 41.119104892879946}
{'ner': 39.06847978921857}
{'ner': 45.09356489178446}
{'ner': 38.42455048227835}
{'ner': 40.385721669434574}
{'ner': 39.44253195279021}
{'ner': 40.88345741156089}
{'ner': 43.560654840845224}
{'ner': 45.76254466503033}
{'ner': 47.396569117740775}
{'ner': 45.15786232812085}
{'ner': 43.47111692991539}
{'ner': 45.628841990419886}
{'ner': 41.30862513346701}
{'ner': 41.56561536850381}
{'ner': 46.34630762472614}
{'ner': 43.99628157638456}
{'ner': 43.44690588365188}
{'ner': 40.956613148445804}
{'ner': 39.284664758348335}
{'ner': 42.62590626094006}
{'ner': 46.67610521076655}
{'ner': 45.49042779086087}
{'ner': 52.3021536826299}
{'ner': 42.25701329818075}
{'ner': 41.353209245920254}
{'ner': 44.39794009577605}
{'ner': 43.72735569140428}
{'ner': 30.295663477895157}
{'ner': 39.72535001789995}
{'ner': 37.910332193246035}
{'ner': 39.42211754420969}
{'ner': 46.2497722394

{'ner': 36.77874141582049}
{'ner': 39.60251546490582}
{'ner': 33.43404211741231}
{'ner': 45.325842359200266}
{'ner': 41.43807311420835}
{'ner': 34.368070407530254}
starting iteration: 12
{'ner': 32.67934284399026}
{'ner': 37.11633048337626}
{'ner': 33.93651959202095}
{'ner': 40.78859825198094}
{'ner': 38.508953996179095}
{'ner': 38.93565006459161}
{'ner': 34.982156415748776}
{'ner': 39.06780537867815}
{'ner': 34.29503423688838}
{'ner': 37.54840228869537}
{'ner': 34.80017738043549}
{'ner': 36.15931463590747}
{'ner': 35.483776352470734}
{'ner': 38.57021298347338}
{'ner': 39.74082673353732}
{'ner': 37.866755591651376}
{'ner': 37.76912763587416}
{'ner': 32.67351634867205}
{'ner': 42.18687738695977}
{'ner': 45.77449279897097}
{'ner': 37.74475304978244}
{'ner': 42.25214284077125}
{'ner': 39.46092476842335}
{'ner': 49.49571345523618}
{'ner': 34.54576951872138}
{'ner': 33.49846793882391}
{'ner': 39.37314726408973}
{'ner': 36.91068387612348}
{'ner': 32.75403862670713}
{'ner': 32.212040804817875

...

{'ner': 36.08242736751794}
{'ner': 34.35966767258675}
{'ner': 33.72995865333415}
{'ner': 32.51360557849657}
{'ner': 29.628240875402444}
{'ner': 34.213810374730095}
{'ner': 41.99661553071977}
{'ner': 36.80947306077851}
{'ner': 38.99281714470621}
{'ner': 46.89127188255804}
{'ner': 31.262870576397752}
{'ner': 33.06488027102203}
{'ner': 43.74828897524692}
{'ner': 24.646138835590406}
starting iteration: 24
{'ner': 33.407051551524646}
{'ner': 33.795183786487875}
{'ner': 33.947343586120994}
{'ner': 36.897187338227965}
{'ner': 34.626320027419986}
{'ner': 41.38605680044356}
{'ner': 35.993852663198126}
{'ner': 33.88916628487311}
{'ner': 34.248766842450884}
{'ner': 34.932017403932505}
{'ner': 36.87127569377505}
{'ner': 37.049446782009625}
{'ner': 39.07889041299794}
{'ner': 35.59227238828782}
{'ner': 29.114535797393998}
{'ner': 39.54194059427106}
{'ner': 39.64711618809384}
{'ner': 38.892079935518744}
{'ner': 44.79995596624207}
{'ner': 40.85987098177737}
{'ner': 41.80735002278872}
{'ner': 37.004042

{'ner': 40.87907753403305}
{'ner': 36.340769900488034}
{'ner': 71.19900316620937}
{'ner': 36.41770152391885}
{'ner': 32.35442778654615}
{'ner': 33.934251593958976}
{'ner': 41.90084567476177}
{'ner': 35.537338252390406}
{'ner': 31.01167185595044}
{'ner': 33.16547176669664}
{'ner': 33.92615259423467}
{'ner': 40.06992329239242}
{'ner': 36.17332446734877}
{'ner': 37.79316704594383}
{'ner': 45.31709751519861}
{'ner': 42.02473366547555}
{'ner': 31.290233018184967}
{'ner': 36.526742845396086}
{'ner': 33.52470384776488}
starting iteration: 30
{'ner': 36.06376926552444}
{'ner': 38.27633038023998}
{'ner': 31.08158533044631}
{'ner': 38.51830292436423}
{'ner': 34.634470949729355}
{'ner': 41.89128357913863}
{'ner': 35.7755203096067}
{'ner': 38.535189974705226}
{'ner': 39.6727586397278}
{'ner': 40.58789948254402}
{'ner': 32.43959630724824}
{'ner': 36.56706806893953}
{'ner': 43.19321567751659}
{'ner': 38.6596402558152}
{'ner': 37.14255141967681}
{'ner': 36.57912501605396}
{'ner': 30.97160912456743}
{

{'ner': 34.87466442761699}
{'ner': 35.730183767543295}
{'ner': 32.93767389830119}
{'ner': 36.458818851510465}
{'ner': 28.97869160037596}
{'ner': 32.94038606445675}
{'ner': 37.991221355974155}
{'ner': 32.43175396809214}
{'ner': 31.250522164900516}
{'ner': 34.787319500285975}
{'ner': 37.62034560252837}
{'ner': 37.331646194486936}
{'ner': 41.49308042017844}
{'ner': 35.1850940094134}
{'ner': 33.58368146306776}
{'ner': 40.86428871571388}
{'ner': 38.3597691868751}
{'ner': 38.70970621901065}
{'ner': 33.79719434280572}
{'ner': 40.09107961495273}
{'ner': 39.79273801865861}
{'ner': 37.663882375598014}
{'ner': 36.89453777151759}
starting iteration: 36
{'ner': 26.296820742642424}
{'ner': 27.93577094474888}
{'ner': 43.5859775544393}
{'ner': 39.354837770996326}
{'ner': 42.8242047894107}
{'ner': 44.00436070987785}
{'ner': 28.690535685003663}
{'ner': 33.99656766466419}
{'ner': 31.910369360531305}
{'ner': 31.758848293425377}
{'ner': 33.64379363152152}
{'ner': 32.99488349856824}
{'ner': 36.2781142429363

{'ner': 31.379498432259343}
{'ner': 34.2229936044586}
{'ner': 41.444788528297615}
{'ner': 34.971664765973}
{'ner': 36.21249537364381}
{'ner': 37.89232602517418}
{'ner': 40.12751544619823}
{'ner': 44.831276757679}
{'ner': 40.35190903055004}
{'ner': 30.401726774940542}
{'ner': 35.895129365413396}
{'ner': 30.72591322783003}
{'ner': 35.515371254024245}
{'ner': 36.534077595986496}
{'ner': 40.020184721015085}
{'ner': 30.041125780335896}
{'ner': 39.38453576118906}
{'ner': 33.95890919283063}
{'ner': 41.71518884804474}
{'ner': 36.70561465929772}
{'ner': 45.61870606503556}
{'ner': 35.14480347799836}
{'ner': 36.92703181639394}
{'ner': 30.63450920212843}
{'ner': 29.76341717568972}
{'ner': 40.219299485821814}
{'ner': 39.527543386765366}
{'ner': 33.677940726185895}
starting iteration: 42
{'ner': 27.489906081621694}
{'ner': 35.06365949562592}
{'ner': 32.46038817274065}
{'ner': 28.779714273944965}
{'ner': 27.539509720261606}
{'ner': 34.046072453898155}
{'ner': 38.29499237291103}
{'ner': 40.38554512658

{'ner': 33.601218810691535}
{'ner': 31.159495443208115}
{'ner': 43.09159736162839}
{'ner': 37.11459661269133}
{'ner': 36.06021963828301}
{'ner': 28.838708739743502}
{'ner': 39.59377423896411}
{'ner': 33.529917583347824}
{'ner': 32.47694466951399}
{'ner': 37.06292311235434}
{'ner': 36.9297772769386}
{'ner': 35.61275056555232}
{'ner': 34.07509888723563}
{'ner': 37.22941548685544}
{'ner': 40.38662614073906}
{'ner': 37.30480057994911}
{'ner': 31.846636523254944}
{'ner': 38.84847998785268}
{'ner': 39.49017497973494}
{'ner': 32.0419092059855}
{'ner': 34.021285432581166}
{'ner': 39.22465289823331}
{'ner': 33.911078144796015}
{'ner': 47.05676225216525}
{'ner': 30.344695646606045}
{'ner': 34.905039221066154}
{'ner': 38.90459540184872}
{'ner': 40.013814293795335}
{'ner': 33.36644215751675}
{'ner': 43.4424894146394}
{'ner': 36.82910911723643}
{'ner': 36.94971738740589}
starting iteration: 48
{'ner': 40.314256020500416}
{'ner': 41.11319677620455}
{'ner': 38.9573871499867}
{'ner': 36.10194501248471

In [54]:
nlp.to_disk("age_model")

In [55]:
nlp = spacy.load("age_model")
txt = 'the Building is only 8 Years old and They are reusing the Bricks.'
txt = 'Roof to the barn around 10 years old he stated.'
txt = 'roof covering is about 15 years old and is in average condition.'
txt = 'Roof metal is approaximately 15 - 20 years old'

In [67]:
testdata = load_data('test_labe2.csv')

In [68]:
testdata.head()

,EVENT_ID,DECISION,NOTES
0,5628149,No,06052019105301 - Hb2 Letter by Adjuster # 8982...
1,5683403,Yes,"Email subject Line of Ruiz, William - Subrogat..."
2,5687563,No,07012019120019 - Hb2 Letter by Adjuster # 8814...
3,5690245,No,looks like the Fencing is not included in his ...
4,5690245,No,looks like the Fencing is not included in his ...


In [ ]:
txt = testdata['NOTES'][4]
print(txt)
doc = nlp(txt)
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
def all_ent(txt):
    entities = []
    try:
        doc = nlp(txt)
        for ent in doc.ents:
            entities = entities + [{ent.label_ : ent.text}]
        return(entities)
    except:
        print(txt)
        return(entities)
testdata['entities'] = testdata['NOTES'].apply(all_ent)

# for i in range(len(testdata['NOTES'][4])):
#     txt = testdata['NOTES'][i]
#     print(txt)
#     doc = nlp(txt)
#     for ent in doc.ents:
#         print(ent.text,ent.label_)

In [71]:
testdata.to_csv("test_labeled2.csv")

In [72]:
testdata.tail()


,EVENT_ID,DECISION,NOTES,entities
731,6512209,NaN,INSPECTION COMPLETE?: YESCAUSE OF LOSS CONFIRM...,[]
732,5343220,NaN,"RAMONA, SAID THEY CANNOT FIGURE OUT WHERE T...",[]
733,5343409,NaN,I SPOKE TO INSURED TODAY. SECURED BRIEF R/S FR...,[{'building': 'AT'}]
734,5344355,NaN,ORIGINAL NOTE - EQUIAN: CLAIM NOTES: 5344355 /...,[]
735,5343440,NaN,"INSPECTED LOSS YESTERDAY WITH INSURED, ROBERT,...",[{'building': 'FLOORING'}]
